In [14]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

warnings.filterwarnings("ignore")

In [15]:
##### Functions to maintain side manipulations #####
def writer(time_out=10):
    # Function for uploading dataframes into the google docs
    def google_upload():
        try:
            # Параметры подключения
            scope = [
                "https://spreadsheets.google.com/feeds",
                "https://www.googleapis.com/auth/drive",
            ]

            json_keyfile_path = "./misc/macro-parser-lme-c2f2972b48fc.json"
            excel_file_path = "./data/multiple_dataframes.xlsx"
            spreadsheet_key = "19XJClFWuwVdKx0R0Hw9owa6B6YJtgr9rgwT_0AJta4k"

            # API токен для подключения
            credentials = ServiceAccountCredentials.from_json_keyfile_name(
                json_keyfile_path, scope
            )
            gc = gspread.authorize(credentials)

            # Читаем файл
            xls = pd.ExcelFile(excel_file_path)

            # Пробегаемся по каждому листу книги и скидываем их в гугл докс
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
                d2g.upload(
                    df, spreadsheet_key, 
                    sheet_name, 
                    credentials=credentials,
                    row_names=True)
                print(f"{sheet_name} uploaded!")
                time.sleep(time_out)

            print("Uploading whole file to Google Sheets completed")

        except Exception as error:
            print(error)
            

    # Выгрузка на диск
    def yandex_upload():
        token = pd.read_csv("./misc/token.csv", header=None)
        token = token.iloc[0, 0]

        y = yadisk.YaDisk(token=token)

        try:
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")

        except Exception as error:
            print(error)
            y.remove("/macro_db/database.xlsx")
            y.upload("./data/multiple_dataframes.xlsx", "/macro_db/database.xlsx")

    # Открываем все необходимые фреймы
    lme = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
    kitko = pd.read_excel("./data/kitko_db.xlsx", index_col=0)
    lbma = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)
    nbk = pd.read_excel("./data/nbk_tenge.xlsx", index_col=0)
    shmet = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
    westmetall = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

    usd = pd.read_excel("./data/cb_currency.xlsx", sheet_name="USD", index_col=0)
    eur = pd.read_excel("./data/cb_currency.xlsx", sheet_name="EUR", index_col=0)
    ausd = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Australian_Dollar", index_col=0
    )
    yuan = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="China_Yuan", index_col=0
    )
    pound = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="British_Pound", index_col=0
    )
    tenge = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Kazakhstan_Tenge", index_col=0
    )
    yen = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Japanese_Yen", index_col=0
    )
    swiss = pd.read_excel(
        "./data/cb_currency.xlsx", sheet_name="Swiss_Franc", index_col=0
    )

    cb_metalls = pd.read_excel(
        "./data/cb_metalls.xlsx", sheet_name="cb_metalls", index_col=0
    )
    
    # Мержим всю собранную инфу в локальную "БД"
    with pd.ExcelWriter(
        "./data/multiple_dataframes.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
        engine="xlsxwriter",
    ) as writer:
        lme.to_excel(writer, sheet_name="LME_non_ferrous", index=False)
        kitko.to_excel(writer, sheet_name="KITCO", index=False)
        lbma.to_excel(writer, sheet_name="LBMA", index=False)
        nbk.to_excel(writer, sheet_name="nbk_tenge", index=False)
        shmet.to_excel(writer, sheet_name="SHMET", index=False)
        westmetall.to_excel(writer, sheet_name="LME_westmetall", index=False)
        usd.to_excel(writer, sheet_name="USD", index=False)
        eur.to_excel(writer, sheet_name="EUR", index=False)
        ausd.to_excel(writer, sheet_name="Australian_Dollar", index=False)
        yuan.to_excel(writer, sheet_name="China_Yuan", index=False)
        pound.to_excel(writer, sheet_name="British_Pound", index=False)
        tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge", index=False)
        yen.to_excel(writer, sheet_name="Japanese_Yen", index=False)
        swiss.to_excel(writer, sheet_name="Swiss_Franc", index=False)
        cb_metalls.to_excel(writer, sheet_name="cb_metalls", index=False)

    print("Database uploaded on yandex.disk")

    yandex_upload()
    google_upload()


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw * 1000
    return int(date_raw)


# Session creation via proxy
def get_session(url):
    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            counter += 1
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f"Attempt {counter+1}")

    return response

async def fetch_html(url, session):
    async with session.get(url) as response:
        return await response.text()

----

### This is the main function block ###

In [16]:
################################################################
##########################   LME ###############################
################################################################
# Асинхронная функция
async def lme_selenium_async():
    url = "https://www.lme.com/Metals/Non-ferrous#tabIndex=1"

    service = Service()
    driver = webdriver.Chrome(service=service)

    try:
        driver.maximize_window()
        driver.get(url=url)

        time.sleep(2)

        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )
        driver.execute_script(
            "window.scrollTo(0, window.scrollY + window.innerHeight);"
        )

        time.sleep(2)

        html_code = driver.page_source

        soup = BeautifulSoup(html_code, "html.parser")
        # Вытаскиваем данные из табличного блока
        data_raw = soup.find_all("div", class_="metal-block-row__blocks")

        # Приводим к тексту
        metalls_raw = data_raw[0].text

        # Убираем все лишнее
        metalls_raw = metalls_raw.replace(" ", "").replace("LME", "LME_").split("\n")
        metalls_raw = " ".join(metalls_raw).strip().split(" ")

        # Конфигурируем новый лист по металлам
        metalls_raw = [metalls_raw[i : i + 4] for i in range(0, len(metalls_raw), 4)]

        # Обрезаем до нужной инфы
        metalls_raw = [i[:2] for i in metalls_raw]

        # Приведем цены к числовому формату
        for metall in metalls_raw:
            metall[1] = pd.to_numeric(metall[1])

        # Формируем строку датафрейма по металлам
        dict_list = [{item[0]: item[1]} for item in metalls_raw]

        metall_df = pd.DataFrame()

        for i in dict_list:
            keys = list(i.keys())
            values = list(i.values())
            metall_df[keys[0]] = values

        # Дропаем ненужные колонки
        metall_df.drop(columns=["LME_AluminiumAlloy", "LME_NASAAC"], inplace=True)
        metall_df = metall_df.rename(
            columns={
                "LME_Aluminium": "aluminium",
                "LME_Copper": "copper",
                "LME_Lead": "lead",
                "LME_Nickel": "nickel",
                "LME_Zinc": "zink",
                "LME_Tin": "tin",
            }
        )

        # Достанем в вставим дату
        date_raw = soup.find_all("span", class_="metal-block-container__refreshed-on")
        date_raw = date_raw[0]
        date_raw = str(date_raw).split(">")[1]
        date_raw = date_raw.replace("\xa0", "").replace("</span", "")
        date_raw = pd.to_datetime(date_raw) - pd.Timedelta(days=1)

        metall_df["date"] = date_raw

        # Поменяем порядок колонок
        new_column_order = [
            "date",
            "aluminium",
            "copper",
            "lead",
            "nickel",
            "zink",
            "tin",
        ]
        metall_df = metall_df[new_column_order]

        # Сохраняем полученные результаты
        lme_db = pd.read_excel("./data/LME_db_new.xlsx", index_col=0)
        lme_db = pd.concat([lme_db, metall_df]).reset_index(drop=True)

        # Дропаем дубликаты
        lme_db = lme_db.drop_duplicates(subset=lme_db.columns.to_list()[1:])

        # Сохраняем
        with pd.ExcelWriter(
            "./data/LME_db_new.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            lme_db.to_excel(writer, sheet_name="LME_main")
        
        print("LME_main is done!!!")
        display(lme_db.tail(10))

    except Exception as error:
        print(error)

    finally:
        driver.close()
        driver.quit()


In [17]:
################################################################
#########################   KITCO (main) ########################
################################################################


# Ассинхронный парсер Kitco
async def kitco_parser_async():
    async def page_getter():
        try:
            url = "https://www.kitco.com/price/fixes/london-fix"

            service = Service()
            driver = webdriver.Chrome(service=service)
            # driver.maximize_window()
            driver.get(url=url)
            #time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source
            
            return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()
            
    
    html_code = await page_getter()
    
    try:
        # Работа над полученными данными
        soup = BeautifulSoup(html_code)

        # Выделим данные и предобработаем для получения дневных данных
        day = soup.find_all("div", class_="border")[1]
        for i in day:
            day = list(i.find_all("div"))

        day = day[4:]

        day = [str(i) for i in day]

        day = [i.replace("<div>", "").replace("</div>", "") for i in day]

        day[0] = pd.to_datetime(day[0])

        day[1] = day[1].split("/")
        day[1] = day[1][1].strip().replace(",", "")
        day[1] = pd.to_numeric(day[1])

        day[2] = pd.to_numeric(day[2])

        day[3] = day[3].split("/")
        day[3] = day[3][1].strip().replace(",", "")
        day[3] = pd.to_numeric(day[3])

        day[4] = day[4].split("/")
        day[4] = day[4][1].strip().replace(",", "")
        day[4] = pd.to_numeric(day[4])

        df_row = (
            pd.DataFrame([day], columns=["Date", "Gold", "Silver", "Platinum", "Palladium"])
            .sort_values("Date")
            .reset_index(drop=True)
        )

        # Выделим данные и предобработаем для получения исторических данных
        historical = soup.find_all("div", class_="border")[2]
        for i in historical:
            historical = list(i.find_all("div"))

        historical = historical[5:]

        historical = [str(i) for i in historical]

        historical = [i.replace("<div>", "").replace("</div>", "") for i in historical]
        historical = [item for item in historical if "<div" not in item]

        # Разобьем по дням
        historical = [historical[i : i + 5] for i in range(0, len(historical), 5)]

        # Обработаем данные внутри каждого дня
        for i in historical:
            i[0] = pd.to_datetime(i[0])

            i[1] = i[1].split("/")
            i[1] = i[1][1].strip().replace(",", "")
            i[1] = pd.to_numeric(i[1])

            i[2] = pd.to_numeric(i[2])

            i[3] = i[3].split("/")
            i[3] = i[3][1].strip().replace(",", "")
            i[3] = pd.to_numeric(i[3])

            i[4] = i[4].split("/")
            i[4] = i[4][1].strip().replace(",", "")
            i[4] = pd.to_numeric(i[4])

        df_historical = (
            pd.DataFrame(
                historical, columns=["Date", "Gold", "Silver", "Platinum", "Palladium"]
            )
            .sort_values("Date")
            .reset_index(drop=True)
        )

        df_historical = pd.concat([df_historical, df_row]).reset_index(drop=True)
        kitko_db = pd.read_excel("./data/kitko_db.xlsx", index_col=0)

        kitko_db = (
            pd.concat([kitko_db, df_historical])
            .reset_index(drop=True)
            .drop_duplicates(subset=["Date"])
        )

        with pd.ExcelWriter(
            "./data/kitko_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            kitko_db.to_excel(writer, sheet_name="kitco_metall")

        print("KITKO_main is done!!!")
        display(kitko_db.tail(10))
        
    except Exception as e:
        print(e)

In [18]:
################################################################
#########################   LBMA (KITCO subs) ##################
################################################################


async def lbma_prescious_async():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    async def get_raw_data(url, metall="Default"):
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # scraped_data = get_session(url=url)

        raw_data = pd.read_json(scraped_data.text)
        data = raw_data[["d", "v"]]
        data["v"] = data["v"].apply(lambda x: x[0])
        data["d"] = pd.to_datetime(data["d"])

        data.rename(columns={"d": "Date", "v": metall}, inplace=True)
        data = data.tail(4)

        return data

    try:
        gold = await get_raw_data(url_gold, metall="Gold")
        silver = await get_raw_data(url_silver, metall="Silver")
        platinum = await get_raw_data(url_platinum, metall="Platinum")
        paladium = await get_raw_data(url_paladium, metall="Palladium")

        result_df = (
            gold.merge(silver, on="Date", how="outer")
            .merge(platinum, on="Date", how="outer")
            .merge(paladium, on="Date", how="outer")
        )

        result_df.fillna(value=0, inplace=True)
        result_df = result_df.sort_values(by="Date")
        result_df.reset_index(inplace=True, drop=True)

        historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

        result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

        result.drop_duplicates(inplace=True)

        result = result.sort_values(by="Date")

        result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")

        print("LBMA is done!!!")
        display(result.tail(10))

    except Exception as error:
        print(error)

In [19]:
################################################################
####################   CB_currencies ###########################
################################################################


async def cb_currency():
    async def get_currency(currency):
        current_date = datetime.today().strftime("%d.%m.%Y")

        dict_of_currencies = {
            "USD": "R01235",
            "EUR": "R01239",
            "Australian_Dollar": "R01010",
            "China_Yuan": "R01375",
            "British_Pound": "R01035",
            "Kazakhstan_Tenge": "R01335",
            "Japanese_Yen": "R01820",
            "Swiss_Franc": "R01775",
        }

        url = f"https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={dict_of_currencies[currency]}&UniDbQuery.\
From=01.01.2023&UniDbQuery.To={current_date}"

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        return scraped_data

    def data_reconstruction(scraped_data, name):

        # Получаем датафрейм
        data = pd.read_html(scraped_data.text.replace(",", "."))[0]
        data.columns = data.iloc[1]

        # Преобразуем
        data = data.iloc[2:]
        data = data.rename(
            columns={
                data.columns[0]: "date",
                data.columns[1]: "unit",
                data.columns[2]: "nominal",
            }
        ).reset_index(drop=True)

        # Cкидываем название индекса
        data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
        data["date"] = pd.to_datetime(data["date"], format="%d.%m.%Y")
        data["unit"] = pd.to_numeric(data["unit"])
        data["nominal"] = pd.to_numeric(data["nominal"])

        data = data.sort_values("date").reset_index(drop=True)

        print(f"{name} is done!")
        display(data.tail())

        return data

    try:
        raw_usd = await get_currency(currency="USD")
        raw_eur = await get_currency(currency="EUR")
        raw_ausd = await get_currency(currency="Australian_Dollar")
        raw_yuan = await get_currency(currency="China_Yuan")
        raw_pound = await get_currency(currency="British_Pound")
        raw_tenge = await get_currency(currency="Kazakhstan_Tenge")
        raw_yen = await get_currency(currency="Japanese_Yen")
        raw_swiss = await get_currency(currency="Swiss_Franc")

        usd = data_reconstruction(raw_usd, name="USD")
        eur = data_reconstruction(raw_eur, name="EUR")
        ausd = data_reconstruction(raw_ausd, name="Australian_Dollar")
        yuan = data_reconstruction(raw_yuan, name="China_Yuan")
        pound = data_reconstruction(raw_pound, name="British_Pound")
        tenge = data_reconstruction(raw_tenge, name="Kazakhstan_Tenge")
        yen = data_reconstruction(raw_yen, name="Japanese_Yen")
        swiss = data_reconstruction(raw_swiss, name="Swiss_Franc")

        with pd.ExcelWriter(
            "./data/cb_currency.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            usd.to_excel(writer, sheet_name="USD")
            eur.to_excel(writer, sheet_name="EUR")
            ausd.to_excel(writer, sheet_name="Australian_Dollar")
            yuan.to_excel(writer, sheet_name="China_Yuan")
            pound.to_excel(writer, sheet_name="British_Pound")
            tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge")
            yen.to_excel(writer, sheet_name="Japanese_Yen")
            swiss.to_excel(writer, sheet_name="Swiss_Franc")

    except Exception as error:
        print(error)
        

In [20]:
################################################################
####################   CB_metalls ##############################
################################################################


async def cb_metalls():
    try:
        # Первичные гиперпараметры
        current_year = datetime.now().year

        url = f"https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted\
=True&UniDbQuery.From=01.01.2023&UniDbQuery.To=31.12.{current_year}&UniDbQuery.Gold\
=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium\
=true&UniDbQuery.so=1"

        # Запуск скраппера
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # Первичное получение данных
        metalls = pd.read_html(scraped_data.text.replace(",", "."))[0]

        # Переименовывание колонок
        metalls = metalls.rename(
            columns={
                metalls.columns[0]: "date",
                metalls.columns[1]: "gold",
                metalls.columns[2]: "silver",
                metalls.columns[3]: "platinum",
                metalls.columns[4]: "palladium",
            }
        )

        # Приведение данных к необходимому формату
        metalls["date"] = pd.to_datetime(metalls["date"], format="%d.%m.%Y")
        metalls["gold"] = pd.to_numeric(metalls["gold"].str.replace(" ", ""))
        metalls["silver"] = pd.to_numeric(metalls["silver"])
        metalls["platinum"] = pd.to_numeric(metalls["platinum"].str.replace(" ", ""))
        metalls["palladium"] = pd.to_numeric(metalls["palladium"].str.replace(" ", ""))

        metalls = metalls.sort_values("date").reset_index(drop=True)

        with pd.ExcelWriter(
            "./data/cb_metalls.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            metalls.to_excel(writer, sheet_name="cb_metalls")

        print("CB_metalls parsing is DONE!")
        display(metalls.tail(20))

    except Exception as error:
        print(error)


In [21]:
################################################################
#########################   NBK ################################
################################################################

async def nbk_tenge_async():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    try:
        year = date.today().year

        upper_bound = f'01.01.2022'
        lower_bound = f'31.12.{year}'

        url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
            /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

        counter = 0

        while counter <= 6:
            try:
                with time_limit(15):
                    page = requests.get(url=url)
                    break

            except TimeoutException:
                # print("NBK_tenge timed out! Another attempt")
                counter += 1

        temp_df = pd.read_html(page.text)
        df = temp_df[0]
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])  # , dayfirst=True)
        df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

        with pd.ExcelWriter(
                './data/nbk_tenge.xlsx') as writer:
            df.to_excel(writer, sheet_name='tenge')

        print('NBK_tenge parsing is DONE!')
        display(df.tail(10))

        return df
    
    except Exception as error:
        print(error)

In [22]:
################################################################
##########################  SHMET ##############################
################################################################

async def shmet_optimized_async():
    url = "https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0"
    
    try:
        responce = requests.get(url)

        day_df = pd.DataFrame(responce.json()["data"])
        day_df["date"] = date.today()
        cooper_row = day_df[day_df["name"].str.contains("cu", case=False)]

        result = cooper_row[["date", "middle", "unit"]]
        result["date"] = pd.to_datetime(result["date"])
        result = result.rename(columns={"middle": "price"})

        hist_data = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)

        new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
        new_df.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            new_df.to_excel(writer, sheet_name="SHMET")

        final = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
        final.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/shmet_historical.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final.to_excel(writer, sheet_name="SHMET")

        print("SHMET is done!!!")
        display(final.head(10))
    
    except Exception as error:
        print(error)


In [23]:
################################################################
################## NEW_WESTMETALL###############################
################################################################


async def new_westmetall_async():
    async def get_data(metall, col_name):
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data["date"] = pd.to_datetime(data["date"])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    try:
        al = await get_data(metall="Al", col_name="aluminium")
        cu = await get_data(metall="Cu", col_name="copper")
        pb = await get_data(metall="Pb", col_name="lead")
        ni = await get_data(metall="Ni", col_name="nickel")
        zn = await get_data(metall="Zn", col_name="zink")
        tn = await get_data(metall="Sn", col_name="tin")

        result = (
            pd.merge(al, cu, on="date", how="left")
            .merge(pb, on="date", how="left")
            .merge(ni, on="date", how="left")
            .merge(zn, on="date", how="left")
            .merge(tn, on="date", how="left")
        )

        old_data = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

        final_data = pd.concat([old_data, result], axis=0)

        final_data.drop_duplicates(subset="date", inplace=True)

        final_data.sort_values(by="date", inplace=True)

        final_data.reset_index(inplace=True, drop=True)

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final_data.to_excel(writer, sheet_name="LME_westmetall")

        print("WESTMETALL is done!!!")
        display(final_data.tail(10))

    except Exception as error:
        print(error)


In [24]:
if __name__ == "__main__":

    await asyncio.gather(
        kitco_parser_async(),
        lbma_prescious_async(),
        lme_selenium_async(),
        new_westmetall_async(),
        cb_currency(),
        cb_metalls(),
        nbk_tenge_async(),
        shmet_optimized_async(),
    )
    
    # time.sleep(10)
    # Аплоад
    #writer(time_out=7)

KITKO_main is done!!!


,Date,Gold,Silver,Platinum,Palladium
391,2024-07-22,2392.70,29.07,948.0,901.0
392,2024-07-23,2403.10,29.06,950.0,911.0
393,2024-07-24,2421.45,29.29,958.0,941.0
394,2024-07-25,2364.20,27.99,935.0,902.0
395,2024-07-26,2386.10,27.75,936.0,908.0
396,2024-07-29,2391.10,28.14,942.0,912.0
397,2024-07-30,2390.25,27.88,957.0,893.0
398,2024-07-31,2426.30,28.56,972.0,923.0
399,2024-08-01,2454.55,28.88,974.0,930.0
400,2024-08-02,2469.85,28.95,972.0,904.0


LBMA is done!!!


,Date,Gold,Silver,Platinum,Palladium
14311,2024-07-22,2392.70,29.065,948.0,901.0
14312,2024-07-23,2403.10,29.055,950.0,911.0
14313,2024-07-24,2421.45,29.290,958.0,941.0
14314,2024-07-25,2364.20,27.990,935.0,902.0
14315,2024-07-26,2386.10,27.755,936.0,908.0
14316,2024-07-29,2391.10,28.140,942.0,912.0
14317,2024-07-30,2390.25,27.880,957.0,893.0
14318,2024-07-31,2426.30,28.560,972.0,923.0
14319,2024-08-01,2454.55,28.880,974.0,930.0
14320,2024-08-02,2469.85,28.950,972.0,904.0


LME_main is done!!!


...

,date,gold,silver,platinum,palladium
369,2024-07-09,6743.87,86.68,2899.89,2908.39
370,2024-07-10,6724.41,87.68,2852.00,2866.14
371,2024-07-11,6688.38,87.72,2813.31,2833.08
372,2024-07-12,6745.03,87.51,2820.39,2814.73
373,2024-07-13,6796.33,87.49,2840.74,2820.99
374,2024-07-16,6794.73,86.73,2800.50,2729.92
375,2024-07-17,6872.34,87.25,2815.64,2713.46
376,2024-07-18,6919.31,87.41,2781.09,2659.31
377,2024-07-19,7007.35,87.34,2881.76,2743.33
378,2024-07-20,6972.37,86.23,2818.61,2688.43


NBK_tenge parsing is DONE!


,date,Числовое значение,ДОЛЛАР США
938,2024-07-27,1,474.83
939,2024-07-28,1,474.83
940,2024-07-29,1,474.83
941,2024-07-30,1,473.29
942,2024-07-31,1,475.37
943,2024-08-01,1,473.95
944,2024-08-02,1,474.71
945,2024-08-03,1,474.57
946,2024-08-04,1,474.57
947,2024-08-05,1,474.57


SHMET is done!!!


,date,price,unit
0,2024-08-05,73050,Yuan/MT
1,2024-08-03,73380,Yuan/MT
2,2024-08-02,74860,Yuan/MT
3,2024-08-01,73810,Yuan/MT
4,2024-07-31,73170,Yuan/MT
5,2024-07-30,74150,Yuan/MT
6,2024-07-27,74275,Yuan/MT
7,2024-07-26,73210,Yuan/MT
8,2024-07-25,74670,Yuan/MT
9,2024-07-24,75480,Yuan/MT


In [25]:
# writer()